## 1. Loading preprocessed dataset

In [1]:
import pandas as pd
X_tr = pd.read_csv("X_tr.csv")
Y_tr = pd.read_csv("Y_tr.csv", header=None)

In [2]:
print("X_tr.shape: {}\nY_tr.shape: {}".format(X_tr.shape, Y_tr.shape))

X_tr.shape: (631760, 113)
Y_tr.shape: (631760, 1)


# 2. Creating, fitting and evaluating the models

Methodology used:
* For each model it will be run 10 resamplings, 10 train/test splits with fitting and evaluation of the model;
* The performance metric adopted for evaluating models in the competition was the f1-score so it will be used here too. As our performance metric is a random variable due to the resampling, it has a mean and standard deviation, making necessary the creation of an objective function with the statistics of the random variable:


$$
\text{maximize} \quad f_o(u) = \mu (u) - 2\sigma(u)
$$

So "best" will mean higher $f_o(u)$. We can say that the objective function tries to balance the two objectives: higher mean, or better generalization, with a lower standard deviation, or lower instability.


* Depending on time complexity some hyperparameter optimization was done.

In [3]:
# Auxiliary function to tell when processing is over
def is_over(): # linux os
    import os
    os.system('spd-say "your program has finished"')

In [4]:
# Function for scaling numerical features
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def scale_feat(X_train, X_test, featIndex, scaleType='min-max'):
    if scaleType=='min-max' or scaleType=='std':
        X_tr_norm = np.copy(X_train) # making a copy to let original available
        X_ts_norm = np.copy(X_test)
        scaler = MinMaxScaler() if scaleType=='min-max' else StandardScaler()
        scaler.fit(X_tr_norm[:,featIndex])
        X_tr_norm[:,featIndex] = scaler.transform(X_tr_norm[:,featIndex])
        X_ts_norm[:,featIndex] = scaler.transform(X_ts_norm[:,featIndex])
        return (X_tr_norm, X_ts_norm)
    else:
        raise ValueError("Type of scaling not defined. Use 'min-max' or 'std' instead.")

In [5]:
import numpy as np

# Hyperparameters:
# Numerical/Ordinal feautures
numFeat = [
    'building_id', # searching for data leakage
    'vdcmun_id',   # categorical, but used as numerical for simplicity
    'ward_id',     # categorical, but used as numerical for simplicity
    'count_floors_pre_eq',
    'count_floors_post_eq',
    'age_building',
    'plinth_area_sq_ft',
    'height_ft_pre_eq',
    'height_ft_post_eq',
    'count_families'
]

# Train/Test split = 80% train and 20% test
test_size = 0.2

# Index of columns to be scaled
numFeat_idx = np.in1d(X_tr.columns.values, numFeat).nonzero()[0]

# Number of resamplings
n_resamplings = 10

In [6]:
# Objective function
def f_o(u):
    return np.mean(u) - 2*np.std(u)

In [7]:
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

## 3.1 k-NN

In [11]:
%%time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

header = ["k", "\mu", "\sigma", "f_o"]
ks = [1, 5, 10] # possible values of k

nn_data = np.zeros((len(ks), len(header)))
for i in range(len(ks)):
    print("Start of k={} at {}".format(ks[i], datetime.datetime.now()))
    results = [0]*n_resamplings
    for j in range(n_resamplings):
        # Train/validation split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, test_size=test_size)

        # scaling
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType='min-max')

        # model fitting
        k_nn = KNeighborsClassifier(n_neighbors=ks[i], n_jobs=-1)
        k_nn.fit(X_tr_norm, y_train)

        # model evaluation
        y_pred = k_nn.predict(X_ts_norm)
        results[j] = f1_score(y_test, y_pred, average='weighted')
        
    
    nn_data[i,:] = np.matrix([ks[i], np.mean(results), np.std(results), f_o(results)])

df_knn = pd.DataFrame(nn_data, columns=header)
display(df_knn)
is_over()

# CPU times: user 3d 11h 17min 47s, sys: 3min, total: 3d 11h 20min 47s
# Wall time: 13h 29min 2s

Start of k=1 at 2019-06-17 00:08:35.281779


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/

Start of k=5 at 2019-06-17 03:17:02.803595


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/

Start of k=10 at 2019-06-17 07:56:13.396357


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/home/romulo/ml_challenge6/venv/

,k,\mu,\sigma,f_o
0,1.0,0.700718,0.001065,0.698588
1,5.0,0.724440,0.001197,0.722045
2,10.0,0.731552,0.001212,0.729127


CPU times: user 3d 11h 17min 47s, sys: 3min, total: 3d 11h 20min 47s
Wall time: 13h 29min 2s


In [21]:
# saving results
# filename = "df_knn.csv"
# df_knn.to_csv(filename, sep='\t', index=False)

# loading results
temp = pd.read_csv("df_knn.csv", sep='\t')
temp

,k,\mu,\sigma,f_o
0,1.0,0.700718,0.001065,0.698588
1,5.0,0.724440,0.001197,0.722045
2,10.0,0.731552,0.001212,0.729127


## 3.2 Linear regression

Regression was done and max value of prediction was the class.
# Ajeitar essa putaria

In [8]:
# Creating the multilabel version of Y_tr
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
Y_tr_multilabel = mlb.fit_transform(Y_tr.values) 
Y_tr_multilabel

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]])

In [77]:
%%time
from sklearn import linear_model

scales = ["min-max", "std"]
header = ["scaleType", "\mu", "\sigma", "f_o"]
reg_data = np.empty((len(scales), len(header)), dtype=object)

for i in range(len(scales)):
    results = [0]*n_resamplings
    for j in range(n_resamplings):
        # Train/validation split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr_multilabel, test_size=test_size)

        # scaling
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType=scales[i])

        # model fitting
        reg = linear_model.LinearRegression(n_jobs=-1)
        reg.fit(X_tr_norm, y_train)

        # model evaluation
        y_pred = np.argmax(reg.predict(X_ts_norm), axis=1).astype(str)
        y_pred = mlb.fit_transform(y_pred)

        results[j] = f1_score(y_test, y_pred, average='weighted')


    reg_data[i,:] = np.matrix([scales[i], np.mean(results), np.std(results), f_o(results)])

df_reg = pd.DataFrame(reg_data, columns=header)
display(df_reg)
is_over()

,scaleType,\mu,\sigma,f_o
0,min-max,0.7072922112223142,0.0014019795563793534,0.7044882521095556
1,std,0.7071527207309989,0.0009284402735831966,0.7052958401838325


CPU times: user 13min 5s, sys: 1min 31s, total: 14min 36s
Wall time: 2min 55s


## 3.3 Logistic regression

In [9]:
%%time
from sklearn import linear_model

penalties = ['l1', 'l2']
Cs = [10**x for x in range(-1,2)]

header = ["penalty", "$C$", "$\mu$", "$\sigma$", "$f_o$"]
logreg_data = np.empty((len(penalties)*len(Cs), len(header)), dtype=object)
count=0
for penalty in penalties:
    for C in Cs:
        print("Started penalty={}/C={} at {}".format(penalty, C, datetime.datetime.now()))
        results = [0]*n_resamplings
        for i in range(n_resamplings):
            # Train/validation split
            X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, test_size=test_size)

            # scaling
            X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType='min-max')

            # model fitting
            logreg = linear_model.LogisticRegression(penalty=penalty, C=C, n_jobs=-1)
            logreg.fit(X_tr_norm, y_train)

            # model evaluation
            y_pred = logreg.predict(X_ts_norm)
            results[i] = f1_score(y_test, y_pred, average='weighted')

        logreg_data[count,:] = np.matrix([penalty, C, np.mean(results), np.std(results), f_o(results)])
        count+=1

df_logreg = pd.DataFrame(logreg_data, columns=header)
display(df_logreg)
is_over()

# CPU times: user 6h 14min 34s, sys: 1min 13s, total: 6h 15min 48s
# Wall time: 6h 14min 56s

Started penalty=l1/C=0.1 at 2019-06-18 01:20:05.649779


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jo

/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jo

Started penalty=l1/C=1 at 2019-06-18 03:33:23.717400


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jo

...

Started penalty=l2/C=10 at 2019-06-18 07:12:01.136772


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jo

/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jo

,penalty,$C$,$\mu$,$\sigma$,$f_o$
0,l1,0.1,0.7157756926989298,0.0010303893451473502,0.7137149140086351
1,l1,1,0.7167765633973547,0.0011240452446798352,0.714528472907995
2,l1,10,0.7164406765307586,0.0007901542904724978,0.7148603679498136
3,l2,0.1,0.7146977404738397,0.0007949291806310744,0.7131078821125776
4,l2,1,0.7165689836654742,0.0010935546911178449,0.7143818742832385
5,l2,10,0.7156428782087111,0.0008206311508626554,0.7140016159069857


CPU times: user 6h 14min 34s, sys: 1min 13s, total: 6h 15min 48s
Wall time: 6h 14min 56s


In [10]:
# saving results
# df_logreg.to_csv("df_logreg.csv", sep='\t', index=False)

# loading results
temp = pd.read_csv("df_logreg.csv", sep='\t')
temp

,penalty,$C$,$\mu$,$\sigma$,$f_o$
0,l1,0.1,0.715776,0.001030,0.713715
1,l1,1.0,0.716777,0.001124,0.714528
2,l1,10.0,0.716441,0.000790,0.714860
3,l2,0.1,0.714698,0.000795,0.713108
4,l2,1.0,0.716569,0.001094,0.714382
5,l2,10.0,0.715643,0.000821,0.714002


## 3.4 Nearest Centroid Classifier (NCC)¶

In [18]:
# Creating a class to represent the classifier
from numpy.linalg import norm

class NearestCentroid():
    def __init__(self):
        self.class_list = {}
        self.centroids = {}
    
    def fit(self, X, y):
        self.class_list = np.unique(y, axis=0) # list of classes
        
        # Basically calculates centroids per class
        self.centroids = np.zeros((len(self.class_list), X.shape[1])) # each row is a centroid
        for i in range(len(self.class_list)): # for each class we evaluate its centroid
            temp = np.where(y==self.class_list[i][0])[0]
            self.centroids[i,:] = np.mean(X[temp],axis=0)
            
            
    def predict(self, X):
        N = len(X)
        y_pred = np.zeros(N)
        for i in range(N):
            # find closest
            dist = np.inf
            closest = None
            for j in range(len(self.class_list)):
                temp = X[i] - self.centroids[j]
                new_dist = np.dot(temp,temp)
                if new_dist < dist:
                    dist = new_dist
                    closest = self.class_list[j][0]
            
            y_pred[i] = closest
        
        return y_pred

In [20]:
%%time
scales = ['min-max', 'std']
header = ["scaleType", "$\mu$", "$\sigma$", "$f_o$"]

ncc_data = np.empty((len(scales), len(header)), dtype=object)

count = 0
for scale in scales:
    print("Started scaleType='{}' at {}".format(scale, datetime.datetime.now()))
    results = [0]*n_resamplings
    for i in range(n_resamplings):
        # Train/validation split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, test_size=test_size)
        
        # scaling
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType=scale)
        
        # model fitting
        ncc = NearestCentroid()
        ncc.fit(X_tr_norm, y_train)
        
        # model evaluation
        y_pred = ncc.predict(X_ts_norm)
        results[i] = f1_score(y_test, y_pred, average='weighted')
        
    ncc_data[count,:] = np.matrix([scale, np.mean(results), np.std(results), f_o(results)])
    count+=1
    

df_ncc = pd.DataFrame(ncc_data, columns=header)
display(df_ncc)
is_over()

Started scaleType=min-max at 2019-06-18 13:58:02.103066
Started scaleType=std at 2019-06-18 13:58:37.553955


,scaleType,$\mu$,$\sigma$,$f_o$
0,min-max,0.6473087562241521,0.0011813360050408248,0.6449460842140704
1,std,0.6320433629645053,0.0012765606173855224,0.6294902417297342


CPU times: user 56.7 s, sys: 14.8 s, total: 1min 11s
Wall time: 1min 11s


## 3.5 Quadratic Gaussian Classifier (QGC)¶

Durante a implementação do CQG ocorreu um problema: o posto das matrizes de covariância das classes é menor do que a quantidade de atributos dos dados, o que foi verificado com o código a seguir.

In [38]:
%%time
from numpy.linalg import matrix_rank

# Train/validation split
X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, test_size=test_size)
X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType=scale)

# Cálculo das médias
class_list = np.unique(y_train, axis=0) # list of classes
# centroids per class
centroids = np.zeros((len(class_list), X_tr_norm.shape[1])) # each row is a centroid
for i in range(len(class_list)): # for each class we evaluate its centroid
    temp = np.where(y_train==class_list[i][0])[0]
    centroids[i,:] = np.mean(X_tr_norm[temp],axis=0)

# Cálculo das matrizes de covariância
C_list = [np.zeros((X_tr_norm.shape[1],X_tr_norm.shape[1]))]*len(class_list)
for i in range(len(class_list)):
    # points of class class_list[i]:
    temp = X_tr_norm[np.where(y_train==class_list[i][0])[0]]
    for datapoint in temp:
        aux = np.asmatrix(datapoint - centroids[i])
        C_list[i] += np.matmul(aux.T,aux)
    C_list[i] /= len(temp) # divide by number of samples


print("Number of features: {}".format(X_tr_norm.shape[1]))
for i in range(len(C_list)):
    print("Rank of C_{} = {}".format(i, matrix_rank(C_list[i])))
    

Number of features: 113
Rank of C_0 = 112
Rank of C_1 = 112
Rank of C_2 = 112
Rank of C_3 = 112
Rank of C_4 = 112
CPU times: user 25 s, sys: 647 ms, total: 25.7 s
Wall time: 25.6 s


Portanto foram utilizadas as seguintes metodologias para tornar as matrizes invertíveis:
 
* Variante 1: extrair apenas a diagonal principal das matrizes;
* Variante 2: matriz de covariância agregada (*pooled*);
* Variante 3: regularização de Friedman, nesse caso foi realizada um busca em grade para determinar o hiper-parâmetro $\lambda$.

In [81]:
from numpy.linalg import inv
from numpy import dot, matmul

class QuadraticGaussianClassifier():
    def __init__(self, variant=None, Lambda=None):
        self.variant = variant
        self.Lambda = Lambda
        self.class_list = {}
        self.centroids = {}
        self.C_inv = {}
        
    
    def fit(self, X, y):
        self.class_list = np.unique(y, axis=0) # list of classes
        
        # Basically calculates centroids per class
        self.centroids = np.zeros((len(self.class_list), X.shape[1])) # each row is a centroid
        for i in range(len(self.class_list)): # for each class we evaluate its centroid
            temp = np.where(y==self.class_list[i][0])[0]
            self.centroids[i,:] = np.mean(X[temp],axis=0)
            
        # Cálculo das matrizes de covariância
        C_list = [np.zeros((X_tr_norm.shape[1],X_tr_norm.shape[1]))]*len(class_list)
        for i in range(len(class_list)):
            # points of class class_list[i]:
            temp = X[np.where(y==self.class_list[i][0])[0]]
            for datapoint in temp:
                aux = np.asmatrix(datapoint - centroids[i])
                C_list[i] += np.matmul(aux.T,aux)
            C_list[i] /= len(temp) # divide by number of samples
        
        if self.variant is None:
            self.C_inv = [inv(C) for C in C_list]
        
        elif self.variant==1: # diagonal
            self.C_inv = [inv(np.diag(np.diag(C))) for C in C_list]
        
        elif self.variant==2: # pooled
            self.C_inv = inv(self.__C_pool(X, y))
        
        elif self.variant==3: # Friedman regularization
            self.C_inv = [np.zeros((X.shape[1],X.shape[1]))]*len(self.class_list)
            N = len(y) # total number of samples
            for i in range(len(self.class_list)): # for each class
                Ni = len(X[np.where(y==self.class_list[i][0])[0]]) # number of samples of class 'i'
                self.C_inv[i] = inv(
                    ((1-self.Lambda)*Ni*C_list[i] + self.Lambda*N*self.__C_pool(X, y)) /
                    ((1-self.Lambda)*Ni           + self.Lambda*N) 
                )
            
    def __C_pool(self, X, y):
        C_poll = np.zeros((X.shape[1],X.shape[1]))
        for i in range(len(self.class_list)):
            # C_poll = sum (Ni/N) * Ci
            C_poll += ( len(X[np.where(y==self.class_list[i][0])[0]]) / len(y) ) * C_list[i]
        
        return C_poll
            
    def predict(self, X):
        N = len(X)
        y_pred = np.zeros(N)
        for i in range(N): # for each sample
            # find closest
            dist = np.inf
            closest = None
            for j in range(len(self.class_list)): # calculate distance from each class
                temp = np.asmatrix(X[i] - self.centroids[j])
                # In variant 2 we have only one covariance matrix, in the other cases we have one covariance
                # matrix for each class:
                C_inv = self.C_inv if variant==2 else self.C_inv[j]
                new_dist = np.matmul(np.matmul(temp,C_inv), temp.T)
                if new_dist < dist:
                    dist = new_dist
                    closest = self.class_list[j][0]
            
            y_pred[i] = closest
                        
        return y_pred

In [124]:
%%time

n_resamplings=10

variants = [1, 2, 3]
lambdas = np.linspace(0,1,num=12)[1:-1] # we crop 0 and 1 because:
                                        # lambda=0 => original QGC
                                        # lambda=1 => variant 2 of QGC
header = ["variant","$\lambda$", "$\mu$", "$\sigma$", "$f_o$"]
qgc_data = np.empty((2 + len(lambdas), len(header)), dtype=object)

for variant in variants:
    print("Started variant {} at {}".format(variant, datetime.datetime.now()))
    results = [0]*n_resamplings if variant!=3 else np.zeros((len(lambdas),n_resamplings))
    for i in range(n_resamplings):
        # Train/validation split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, test_size=test_size)
        
        # scaling
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType="min-max")
        
        if variant!=3:
            # model fitting
            qgc = QuadraticGaussianClassifier(variant=variant)
            qgc.fit(X_tr_norm, y_train)
            
            # model evaluation
            y_pred = qgc.predict(X_ts_norm)
            results[i] = f1_score(y_test, y_pred, average='weighted')
            
        else:
            for j in range(len(lambdas)): # for each lambda
                # model fitting
                qgc = QuadraticGaussianClassifier(variant=variant, Lambda=lambdas[j])
                qgc.fit(X_tr_norm, y_train)
                
                # model evaluation
                y_pred = qgc.predict(X_ts_norm)
                results[j,i] = f1_score(y_test, y_pred, average='weighted')
        
    if variant!=3:
        qgc_data[variant-1,:] = np.asmatrix(
            [variant, np.nan, np.mean(results), np.std(results), f_o(results)]
        )

    else:
        var3_matrix = np.asmatrix([3]*len(lambdas)).T
        lambdas_matrix  = np.asmatrix(lambdas).T
        fo = [f_o(result) for result in results]
        fo = np.asmatrix(fo).T
        qgc_data[2:2+len(lambdas),:] = np.concatenate(
            (var3_matrix, lambdas_matrix, np.asmatrix(np.mean(results,axis=1)).T, 
             np.asmatrix(np.std(results,axis=1)).T, fo), axis=1
        )


df_qgc = pd.DataFrame(qgc_data, columns=header)
display(df_qgc)
is_over()

Started variant 1 at 2019-06-18 21:06:56.687818


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

Started variant 2 at 2019-06-18 21:17:35.235734


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

Started variant 3 at 2019-06-18 21:28:02.312084


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

,variant,$\lambda$,$\mu$,$\sigma$,$f_o$
0,1,NaN,0.635737,0.00076792,0.634201
1,2,NaN,0.509072,0.00153074,0.506011
2,3,0.0909091,0.63296,0.000847989,0.631264
3,3,0.181818,0.63289,0.00087088,0.631148
4,3,0.272727,0.63285,0.000848448,0.631153
5,3,0.363636,0.632503,0.000859601,0.630784
6,3,0.454545,0.631791,0.000819509,0.630152
7,3,0.545455,0.629895,0.000721132,0.628453
8,3,0.636364,0.627478,0.000774536,0.625929
9,3,0.727273,0.616343,0.000784559,0.614774


CPU times: user 6h 20min 15s, sys: 4h 36min 1s, total: 10h 56min 17s
Wall time: 2h 7min 30s


In [8]:
# saving results
#df_qgc.to_csv("df_qgc.csv", sep='\t', index=False)

# loading results
#temp = pd.read_csv("df_qgc.csv", sep='\t')
#temp

## 3.6 Decison trees

In [8]:
%%time
from xgboost import XGBClassifier
import datetime

cases = [
    {
    "learning_rate"     : learning_rate
    ,'n_estimators'     : n_estimators
    ,'max_depth'        : max_depth
    ,'tree_method'      : 'gpu_hist'
    ,'objective'        : 'multi:softmax'
    } 
    # hyperparameters possible values
    for learning_rate    in [1e-1, 0.3, 0.5, 1]
    for n_estimators     in [500, 1000]
    for max_depth        in [6, 12]
]

header = list(cases[0].keys())[:-2] + ["$\mu$", "$\sigma$", "$f_o$"] # no need for repeating
                                                                     # 'tree_method' and 'objective'

xgb_data = np.empty((len(cases), len(header)), dtype=object)
count=0
for case in cases:
    print("Starting instance {}/{} at {}".format(count+1, len(cases), datetime.datetime.now()))
    results = [0]*n_resamplings 
    for i in range(n_resamplings):
        # Train/test split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr.values, 
                                                            test_size=test_size)
        # Train/validation split
        X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, 
                                                                        test_size=test_size)

        # no scaling is needed!!!
        
        # model fitting
        xgb = XGBClassifier(**case)
        xgb.fit(X_train, y_train, eval_set=[(X_validation,y_validation)]
                ,early_stopping_rounds=30, verbose=False)
                
        # model evaluation
        y_pred = xgb.predict(X_test)
        results[i] = f1_score(y_test, y_pred, average='weighted')
            

    xgb_data[count,:] = np.matrix(
        list(case.values())[:-2] + [np.mean(results), np.std(results), f_o(results)])
    count+=1
        
    
df_xgb = pd.DataFrame(xgb_data, columns=header)
display(df_xgb)
is_over()

Starting instance 1/16 at 2019-06-20 15:54:08.632738


/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Starting instance 2/16 at 2019-06-20 17:04:37.899276
Starting instance 3/16 at 2019-06-20 18:53:39.112593
Starting instance 4/16 at 2019-06-20 21:01:29.748893
Starting instance 5/16 at 2019-06-20 22:46:57.437688
Starting instance 6/16 at 2019-06-20 23:53:29.567741
Starting instance 7/16 at 2019-06-21 00:45:48.732393
Starting instance 8/16 at 2019-06-21 01:48:30.566067
Starting instance 9/16 at 2019-06-21 02:39:01.178651
Starting instance 10/16 at 2019-06-21 03:24:14.746130
Starting instance 11/16 at 2019-06-21 03:56:39.753960
Starting instance 12/16 at 2019-06-21 04:41:34.016421
Starting instance 13/16 at 2019-06-21 05:15:35.027745
Starting instance 14/16 at 2019-06-21 05:41:43.127295
Starting instance 15/16 at 2019-06-21 06:03:00.529736
Starting instance 16/16 at 2019-06-21 06:31:54.138944


,learning_rate,n_estimators,max_depth,$\mu$,$\sigma$,$f_o$
0,0.1,500,6,0.755998,0.000845448,0.754307
1,0.1,500,12,0.767047,0.00108983,0.764868
2,0.1,1000,6,0.762722,0.00129213,0.760138
3,0.1,1000,12,0.766463,0.00163742,0.763188
4,0.3,500,6,0.76478,0.00113474,0.762511
5,0.3,500,12,0.765584,0.000913105,0.763758
6,0.3,1000,6,0.764367,0.00134844,0.761671
7,0.3,1000,12,0.766257,0.00111271,0.764031
8,0.5,500,6,0.764135,0.00101605,0.762103
9,0.5,500,12,0.762198,0.000811206,0.760576


CPU times: user 11h 21min 42s, sys: 3h 37min 24s, total: 14h 59min 7s
Wall time: 14h 59min


In [9]:
df_xgb.sort_values("$f_o$", ascending=False).iloc[0,:].to_frame().transpose()

,learning_rate,n_estimators,max_depth,$\mu$,$\sigma$,$f_o$
1,0.1,500,12,0.767047,0.00108983,0.764868


In [8]:
# saving results
#df_xgb.to_csv("df_xgb.csv", sep='\t', index=False)

# loading results
df_xgb = pd.read_csv("df_xgb.csv", sep='\t')
#df_xgb

## 3.7 Artificial Neural Network

In [8]:
%%time 
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K
from keras.callbacks import EarlyStopping

from matplotlib import pyplot


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Y_tr_categorical = to_categorical(Y_tr)[:, 1:] # change to categorical

cases = [
    {
        'lr'  : lr,
        'arq' : arq
    } 
    # hyperparameters possible values
    for arq in [
                 [32],[128],[512]         # MLP with one   layer
                ,[512,128], [128,32]      # //   //  two   layers
                ,[512,128,32], [128,32,8] # //   //  three  //
                ]
    for lr in [1e-3, 1e-2]
]

header = list(cases[0].keys()) + ["$\mu$", "$\sigma$", "$f_o$"] 

ann_data = np.empty((len(cases), len(header)), dtype=object)
count=0
for case in cases:
    print("Started case {}/{} at {}".format(count+1, len(cases),datetime.datetime.now()))

    # model building
    arq = case['arq']
    model = Sequential()
    model.add(Dense(units=arq[0], activation='relu', input_dim=X_tr.shape[1])) # first layer
    for layer in arq:
        model.add(Dense(units=layer, activation='relu'))
    model.add(Dense(units=5, activation='softmax')) # output layer
    
    
    model.compile(loss='categorical_crossentropy'
                 ,metrics=[f1]
                 ,optimizer=Adam(lr=case['lr'], amsgrad=True)
                 )

    results = [0]*n_resamplings
    for i in range(n_resamplings):
        # Train/validation split
        X_train, X_test, y_train, y_test = train_test_split(X_tr.values, Y_tr_categorical, test_size=test_size)

        # scaling
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, featIndex=numFeat_idx, scaleType="min-max")

        # model fitting
        es = EarlyStopping(monitor='val_f1', mode='max', verbose=0, patience=5)
        history = model.fit(X_tr_norm, y_train,shuffle=True 
                            ,epochs=1_000
                            ,verbose=0
                            ,validation_split=0.2
                            ,callbacks=[es]
                            )
    
        # model evaluation
        y_pred = np.argmax(model.predict(X_ts_norm), axis=1)+1
        y_pred = to_categorical(y_pred)[:, 1:]
        results[i] = f1_score(y_test, y_pred, average='weighted')
                
    ann_data[count,:] = np.matrix(list(case.values()) + [np.mean(results), np.std(results), f_o(results)])
    count+=1
        
        
df_ann = pd.DataFrame(ann_data, columns=header)
display(df_ann)
is_over()

Using TensorFlow backend.


Started case 1/14 at 2019-06-22 23:22:22.313922
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Started case 2/14 at 2019-06-23 00:44:14.736540
Started case 3/14 at 2019-06-23 01:50:56.958140
Started case 4/14 at 2019-06-23 02:47:19.652062
Started case 5/14 at 2019-06-23 03:59:12.835073
Started case 6/14 at 2019-06-23 05:00:04.060403
Started case 7/14 at 2019-06-23 06:16:21.475074
Started case 8/14 at 2019-06-23 07:27:20.731248
Started case 9/14 at 2019-06-23 09:06:47.588036
Started case 10/14 at 2019-06-23 10:09:45.965667
Started case 11/14 at 2019-06-23 11:14:46.397510
Started case 12/14 at 2019-06-23 12:38:15.383138
Started case 13/14 at 2019-06-23 15:14:04.314393
Started case 14/14 at 2019-06-23 16:28:36.851462


,lr,arq,$\mu$,$\sigma$,$f_o$
0,0.001,[32],0.736151,0.00208709,0.731977
1,0.01,[32],0.7259,0.00636926,0.713162
2,0.001,[128],0.749068,0.00637921,0.73631
3,0.01,[128],0.732377,0.00422453,0.723928
4,0.001,[512],0.770884,0.0185163,0.733851
5,0.01,[512],0.733888,0.00552421,0.72284
6,0.001,"[512, 128]",0.773487,0.0197325,0.734022
7,0.01,"[512, 128]",0.73511,0.00671424,0.721681
8,0.001,"[128, 32]",0.748759,0.00604128,0.736676
9,0.01,"[128, 32]",0.734137,0.00382495,0.726487


CPU times: user 1d 7h 58min 2s, sys: 3h 33min 4s, total: 1d 11h 31min 6s
Wall time: 18h 41min 29s


In [11]:
# saving results
# df_ann.to_csv("df_ann.csv", sep='\t', index=False)

# loading results
# temp = pd.read_csv("df_ann.csv", sep='\t')
# temp

In [10]:
df_ann.sort_values("$f_o$", ascending=False).iloc[0,:].to_frame().transpose()

,lr,arq,$\mu$,$\sigma$,$f_o$
8,0.001,"[128, 32]",0.748759,0.00604128,0.736676


# 4. Making predictions in the test set

The tree-based model was adopted as it presents better generalization and stability.

In [9]:
temp = df_xgb.sort_values("$f_o$", ascending=False).iloc[0,:][0:3].to_dict()
temp.update({'tree_method':'gpu_hist', 'objective':'multi:softmax'})
best_params = {key:value if key!='n_estimators'and key!='max_depth' else int(value) for key, value in temp.items()}
print("best_params:")
display(best_params)

best_params:


{'learning_rate': 0.1,
 'n_estimators': 500,
 'max_depth': 12,
 'tree_method': 'gpu_hist',
 'objective': 'multi:softmax'}

In [10]:
%%time
# training the chosen model in all the training dataset
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

# Train/validation split
X_train, X_validation, y_train, y_validation = train_test_split(X_tr, Y_tr, test_size=test_size)

# no scaling is needed!!!

# model fitting
xgb = XGBClassifier(**best_params)
xgb.fit(X_train, y_train, eval_set=[(X_validation,y_validation)]
        ,early_stopping_rounds=30, verbose=False)

# model final evaluation on validation set
y_pred = xgb.predict(X_validation)
print("F1_validation: {}".format(f1_score(y_validation, y_pred, average='weighted')))

is_over()

/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/romulo/ml_challenge6/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1_validation: 0.7680552879792706
CPU times: user 8min 23s, sys: 2min 22s, total: 10min 46s
Wall time: 10min 46s


In [11]:
import pickle

with open('model_data.pkl', 'wb') as output:
    pickle.dump(xgb, output, pickle.HIGHEST_PROTOCOL)

with open('model_data.pkl', 'rb') as input:
    xgb = pickle.load(input)

In [12]:
xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=nan, n_estimators=500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [13]:
# preprocessing of the test dataset
test       = pd.read_csv("./dataset/test.csv")
build_own  = pd.read_csv("./dataset/Building_Ownership_Use.csv") 
build_str  = pd.read_csv("./dataset/Building_Structure.csv")
build_data = pd.merge(build_str, build_own,  on=['building_id', 'district_id', 'vdcmun_id', 'ward_id'])
testFull   = pd.merge(test,      build_data, on=['building_id', 'district_id', 'vdcmun_id'])
test_num   = testFull

# encoding to dummies
catFeat = ['area_assesed','district_id','land_surface_condition','foundation_type','roof_type',
           'ground_floor_type','other_floor_type','position','plan_configuration','condition_post_eq',
           'legal_ownership_status']
test_num = pd.get_dummies(test_num, columns=catFeat,                drop_first=True)
test_num = pd.get_dummies(test_num, columns=['has_repair_started'], drop_first=True, dummy_na=True)

# Converting 'building_id' to numerical format
test_num['building_id'] = test_num['building_id'].apply(lambda x: int(x,16))
print("test.shape = {}".format(test_num.shape))

test.shape = (421175, 113)


In [17]:
test_predictions = xgb.predict(test_num)

# Converting predictions to submission format
temp = ['Grade {}'.format(prediction) for prediction in test_predictions]

In [18]:
# creating and saving predictions dataframe for submission
submission = pd.DataFrame(data={'building_id':  testFull['building_id'].values,
                                'damage_grade': temp
                               })

submission.to_csv('submission.csv', index=False)

submission.head()

,building_id,damage_grade
0,a3380c4f75,Grade 4
1,a338a4e653,Grade 5
2,a338a4e6b7,Grade 5
3,a33a6eaa3a,Grade 3
4,a33b073ff6,Grade 5
